### portmy database: stocks table
### stock database: buy, price tables

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

today = date.today()
print(today)

2025-09-28


In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-09-28
yesterday: 2025-09-26 00:00:00


In [4]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 4)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2025-09-22
yesterday: 2025-09-26


### Restart & Run All Cells

In [6]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Weekly
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### This process affects only already in port stocks. To highlight price changes in percent.

In [9]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [10]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_buy = pd.read_sql(sql, const)

names = df_buy.name.tolist()
portfolio = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
    SELECT name, price AS price_d 
    FROM price 
    WHERE date = '{}' AND name IN ({})
    ORDER BY name, date
""".format(yesterday, portfolio)

df_today = pd.read_sql(sql, const)
df_today.shape[0]

29

In [11]:
sql = """
    SELECT name, price AS price_w
    FROM price 
    WHERE date = '{}' AND name IN ({}) 
    ORDER BY name
""".format(week_ago, portfolio)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]

29

In [12]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.tail()

,name,price_d,price_w
24,TFFIF,6.15,6.05
25,TOA,14.00,14.30
26,TVO,25.25,25.25
27,WHAIR,6.05,5.90
28,WHART,9.70,9.45


In [13]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [14]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [15]:
trend.tail()

,name,price_d,price_w,percent
24,TFFIF,6.15,6.05,1.65
25,TOA,14.00,14.30,-2.10
26,TVO,25.25,25.25,0.00
27,WHAIR,6.05,5.90,2.54
28,WHART,9.70,9.45,2.65


In [16]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [17]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
17,PTTGC,26.25,27.25,-3.67%,Worse
5,CPF,22.20,23.00,-3.48%,Worse
4,BCH,13.10,13.50,-2.96%,Worse
25,TOA,14.00,14.30,-2.10%,Worse
2,AIMIRT,10.30,10.50,-1.90%,Worse
9,IVL,22.40,22.80,-1.75%,Worse
20,SENA,1.95,1.97,-1.02%,Worse
18,RCL,27.00,27.25,-0.92%,Worse
3,AWC,2.36,2.38,-0.84%,Worse
22,STA,12.70,12.80,-0.78%,Worse


In [18]:
file_name = 'trend.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [19]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\trend.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\trend.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\trend.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\trend.csv


In [20]:
trend.sort_values(['percent'],ascending=[True]).to_csv(output_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(god_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(icd_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(osd_file, index=False)

### Filter only performance = "Worse"

In [22]:
mask = trend.perform == 'Worse'
trend[mask].tail()

,name,price_d,price_w,percent,perform
17,PTTGC,26.25,27.25,-3.67,Worse
18,RCL,27.00,27.25,-0.92,Worse
20,SENA,1.95,1.97,-1.02,Worse
22,STA,12.70,12.80,-0.78,Worse
25,TOA,14.00,14.30,-2.10,Worse


In [23]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,44.83%
Worse,37.93%
No Change,17.24%


In [24]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(246, 8)

In [25]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [26]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).tail()

,name,price_d,price_w,percent,perform,max,min,pe,pbv,volume,beta,market,mrkt
13,NER,4.30,4.20,2.38,Better,7.65,5.40,6.05,1.93,51.38,0.89,sSET / SETTHSI,SET999
27,WHAIR,6.05,5.90,2.54,Better,9.06,7.02,999.99,0.84,8.83,0.22,SET,SET999
28,WHART,9.70,9.45,2.65,Better,12.60,9.65,999.99,1.07,27.38,0.44,SET,SET999
21,SINGER,7.45,7.20,3.47,Better,59.25,26.25,23.53,1.49,159.28,2.26,SET100 / SETWB,SET100
15,PTG,9.10,8.70,4.60,Better,16.40,12.90,26.07,2.59,187.88,1.24,SET100 / SETTHSI,SET100


In [27]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
17,PTTGC,27.25,26.25,-3.67%,Worse,SET50
5,CPF,23.00,22.20,-3.48%,Worse,SET50
9,IVL,22.80,22.40,-1.75%,Worse,SET50
3,AWC,2.38,2.36,-0.84%,Worse,SET50
16,PTT,33.00,33.25,0.76%,Better,SET50
10,JMART,9.65,9.75,1.04%,Better,SET50
11,JMT,12.30,12.50,1.63%,Better,SET50
19,SCC,222.00,226.00,1.80%,Better,SET50


In [28]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Worse,50.00%
Better,50.00%


In [29]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
4,BCH,13.50,13.10,-2.96%,Worse,SET100
18,RCL,27.25,27.00,-0.92%,Worse,SET100
22,STA,12.80,12.70,-0.78%,Worse,SET100
14,ORI,2.80,2.82,0.71%,Better,SET100
21,SINGER,7.20,7.45,3.47%,Better,SET100
15,PTG,8.70,9.10,4.60%,Better,SET100


In [30]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Worse,50.00%
Better,50.00%


In [31]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
25,TOA,14.30,14.00,-2.10%,Worse,SET999
2,AIMIRT,10.50,10.30,-1.90%,Worse,SET999
20,SENA,1.97,1.95,-1.02%,Worse,SET999
1,AH,14.00,13.90,-0.71%,Worse,SET999
6,CPNREIT,11.40,11.40,0.00%,No Change,SET999
7,DIF,9.10,9.10,0.00%,No Change,SET999
8,GVREIT,6.50,6.50,0.00%,No Change,SET999
12,MCS,8.40,8.40,0.00%,No Change,SET999
26,TVO,25.25,25.25,0.00%,No Change,SET999
0,3BBIF,6.45,6.50,0.78%,Better,SET999


In [32]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,40.00%
No Change,33.33%
Worse,26.67%


In [33]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [34]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,


In [35]:
const.close()
conmy.close()

In [36]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-09-28 22:31:14
